__Compare FWI result with true model for the Overthrust model__

Daniel Köhn 
Kiel, 16/07/2016

__Import Libraries__

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from matplotlib.colors import LightSource, Normalize
from matplotlib.pyplot import gca
from pylab import rcParams
from matplotlib import rc
from matplotlib.ticker import FormatStrFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.ndimage.filters import gaussian_filter
import pickle

__FD grid dimensions__

In [ ]:
DH = 0.2;
NX = 500;
NY = 304;

__Wavefield clip value__ 

In [ ]:
vpmin = 60.0;
vpmax = 800.0;

__Define fonts__

In [ ]:
FSize = 20
font = {'color':  'black',
        'weight': 'normal',
        'size': FSize}
mpl.rc('xtick', labelsize=FSize) 
mpl.rc('ytick', labelsize=FSize) 
rcParams['figure.figsize'] = 16, 9

__Read FWI result and true model__

In [ ]:
f = open("Kleinneudorf_fsurf.vs")
data_type = np.dtype ('float32').newbyteorder ('<')
mod_true = np.fromfile (f, dtype=data_type)
mod_true = mod_true.reshape(NX,NY)
mod_true = np.transpose(mod_true)
mod_true = np.flipud(mod_true)
mod_nosmooth = mod_true

In [ ]:
f = open("Kleinneudorf_fsurf.vs")
data_type = np.dtype ('float32').newbyteorder ('<')
taper = np.fromfile (f, dtype=data_type)
taper = taper.reshape(NX,NY)
taper = np.transpose(taper)
taper = np.flipud(taper)

__Apply Gaussian filter__

In [ ]:
taper = gaussian_filter(taper, sigma=[10,10])
mod_true = gaussian_filter(mod_true, sigma=[15,15])

__Define Axis__

In [ ]:
x = np.arange(0.0, DH*NX, DH)
y = np.arange(0.0, DH*NY, DH)

__Define SubPlot__

In [ ]:
def do_plot(n, model, cm, an, title, vpmin, vpmax):
    
    ax=plt.subplot(2, 1, n)
    
    #plt.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
    rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
    ## for Palatino and other serif fonts use:
    #rc('font',**{'family':'serif','serif':['Palatino']})
    #plt.rc('text', usetex=True)
    rc('text', usetex=True)
    
    im1 = plt.imshow(model, cmap=cm, interpolation='none', extent=[0.0,NX*DH,0.0,NY*DH], vmin=vpmin, vmax=vpmax, aspect='auto')
    a = gca()
    a.set_xticklabels(a.get_xticks(), font)
    a.set_yticklabels(a.get_yticks(), font)
    #plt.axis('scaled')
    plt.title(title, fontdict=font)
    plt.ylabel('Depth [m]', fontdict=font)
    if n==2:
        plt.xlabel('Distance [m]', fontdict=font)
    plt.gca().invert_yaxis()
    if n!=2:
        ax.xaxis.set_major_formatter(plt.NullFormatter())
    
    # add annotation
    #if n!=2:
    #    plt.text(0.5, 4.2,an,fontdict=font,color='white',size=20)
    
    # fit and label colorbar
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="2.5%", pad=0.05)
    cbar = plt.colorbar(im1, cax=cax)
    cbar.set_label(an, fontdict=font, labelpad=3)

__Plot SubPlots__

In [ ]:
plt.close('all')
plt.figure()
do_plot(2, mod_true, 'magma', 'Vp [m/s]', r"Kleinneudorf smooth FATT model", np.min(mod_true), np.max(mod_true))
do_plot(1, mod_nosmooth, 'magma', 'Vp [m/s]', r"Kleinneudorf FATT model", np.min(mod_true), np.max(mod_true))
#plt.savefig('test.png', format='png', dpi=200)
plt.savefig('test.pdf', bbox_inches='tight', format='pdf')
plt.tight_layout()
plt.show()

__Write initial model and taper__

In [ ]:
# P-wave velocity
name_taper = "Kleinneudorf_fsurf_smooth.vs"
f = open (name_taper, mode='wb')
data_type = np.dtype ('float32').newbyteorder ('<')
mod_true = np.array(mod_true, dtype=data_type)
mod_true = np.rot90(mod_true,3)
mod_true.tofile(f)
f.close()

# taper
name_taper = "Kleinneudorf_fsurf_smooth.rho"
f = open (name_taper, mode='wb')
data_type = np.dtype ('float32').newbyteorder ('<')
taper = np.array(taper, dtype=data_type)
taper = np.rot90(taper,3)
taper.tofile(f)
f.close()